<a href="https://colab.research.google.com/github/mariyakj/ASD/blob/main/Brain_Autism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
# Mount Google Drive if your data is stored there
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# Define paths to your training and validation datasets
train_data_dir = '/content/drive/My Drive/mri sample/random train'
val_data_dir = '/content/drive/My Drive/mri sample/random test'

In [ ]:
# Parameters
img_width, img_height = 150, 150
batch_size = 32

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


In [ ]:
val_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

# Load and preprocess validation data
val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 960 images belonging to 2 classes.
Found 297 images belonging to 2 classes.


In [ ]:
# Calculate the steps_per_epoch based on the length of the training generator and batch size
steps_per_epoch_train = train_generator.samples // batch_size

# Similarly, calculate the steps_per_epoch for the validation data
steps_per_epoch_val = val_generator.samples // batch_size

In [ ]:

# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [ ]:

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    epochs=10,
    validation_data=val_generator,
    validation_steps=3
)

Epoch 1/10
30/30 [==============================] - 464s 15s/step - loss: 0.6963 - accuracy: 0.5979 - val_loss: 0.6108 - val_accuracy: 0.7604
Epoch 2/10
30/30 [==============================] - 94s 3s/step - loss: 0.5326 - accuracy: 0.7219 - val_loss: 0.3824 - val_accuracy: 0.8438
Epoch 3/10
30/30 [==============================] - 75s 3s/step - loss: 0.3676 - accuracy: 0.8271 - val_loss: 0.5923 - val_accuracy: 0.7188
Epoch 4/10
30/30 [==============================] - 71s 2s/step - loss: 0.2220 - accuracy: 0.9156 - val_loss: 0.1126 - val_accuracy: 0.9583
Epoch 5/10
30/30 [==============================] - 60s 2s/step - loss: 0.1458 - accuracy: 0.9302 - val_loss: 0.1010 - val_accuracy: 0.9271
Epoch 6/10
30/30 [==============================] - 55s 2s/step - loss: 0.0928 - accuracy: 0.9667 - val_loss: 0.0251 - val_accuracy: 1.0000
Epoch 7/10
30/30 [==============================] - 60s 2s/step - loss: 0.0657 - accuracy: 0.9760 - val_loss: 0.0073 - val_accuracy: 1.0000
Epoch 8/10
30/30 [

In [ ]:

# Save the trained model to an HDF5 file
model.save('/content/drive/My Drive/models/my_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Mount Google Drive if your model is stored there
from google.colab import drive
drive.mount('/content/drive')

# Define the path to your saved model
saved_model_path = '/content/drive/My Drive/models/my_model.h5'

# Load the model
loaded_model = tf.keras.models.load_model(saved_model_path)

# Define the path to the image you want to predict
image_path = '/content/drive/My Drive/mri sample/random test/random mri autism testing/Copy of Y116.JPG'

# Load and preprocess the image
img = image.load_img(image_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize the pixel values

# Make the prediction
prediction = loaded_model.predict(img_array)

# Interpret the prediction
if prediction[0][0] > 0.5:
    print("Prediction: Class 1 (Yes)")
else:
    print("Prediction: Class 0 (No)")
